In [1]:
import pandas as pd 
df=pd.read_csv('dataset/papers.csv',engine='c', error_bad_lines=False)
df.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


In [2]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
stop_words = set(stopwords.words('english'))
new_words = ["fig","figure","image","sample","using","show", "result", "large", "one", "two", "three","four", "five", "seven","eight","nine","also"]
stop_words = list(stop_words.union(new_words))
stop_words

['i',
 'myself',
 'those',
 's',
 "shan't",
 'where',
 'no',
 'they',
 'now',
 'all',
 'shouldn',
 'sample',
 'these',
 'before',
 'him',
 "doesn't",
 "won't",
 'other',
 'or',
 'few',
 'how',
 'y',
 "hadn't",
 'yourself',
 'too',
 "mightn't",
 'than',
 'ourselves',
 'over',
 'any',
 'o',
 'the',
 'nor',
 'mightn',
 'an',
 'eight',
 'seven',
 'if',
 'have',
 'five',
 'some',
 'there',
 'being',
 'off',
 'your',
 'figure',
 're',
 'don',
 "hasn't",
 "couldn't",
 'needn',
 'about',
 "that'll",
 've',
 'that',
 'did',
 'he',
 'against',
 'theirs',
 'itself',
 'at',
 'can',
 'do',
 'didn',
 'own',
 'show',
 'to',
 'then',
 'as',
 'up',
 'is',
 'one',
 "you're",
 'just',
 'ain',
 "weren't",
 'you',
 'four',
 'out',
 'doesn',
 'which',
 'yourselves',
 'be',
 't',
 'above',
 'only',
 'fig',
 'hers',
 'for',
 'she',
 'when',
 'until',
 'very',
 'will',
 'hasn',
 'weren',
 'here',
 'under',
 'again',
 'his',
 'were',
 'me',
 'down',
 'in',
 'ma',
 'using',
 'from',
 'm',
 'through',
 'doing',
 

In [4]:
def preprocessor(text):
    text=text.lower()
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    text=re.sub("(\\d|\\W)+"," ",text) 
    text = text.split()
    text = [word for word in text if word not in stop_words]
    text = [word for word in text if len(word) >= 3]
    lmtzr = WordNetLemmatizer()
    text = [lmtzr.lemmatize(word) for word in text]
    return ' '.join(text)

In [5]:
documents = df['paper_text'].apply(lambda x:preprocessor(x))
documents

0       self organization associative database applica...
1       mean field theory layer visual cortex applicat...
2       storing covariance associative long term poten...
3       bayesian query construction neural network mod...
4       neural network ensemble cross validation activ...
                              ...                        
7236    single transistor learning synapsis paul hasle...
7237    bias variance combination least square estimat...
7238    real time clustering cmos neural engine serran...
7239    learning direction global motion class psychop...
7240    correlation interpolation network real time ex...
Name: paper_text, Length: 7241, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
countvector=CountVectorizer(max_df=0.95,max_features=10000,ngram_range=(1,3))
wcvector=countvector.fit_transform(documents)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
transformer.fit(wcvector)

In [ ]:
def extractVector(featurenames, sorteditems, top=10):
    sorteditems = sorteditems[:top]
    scorevals = []
    featurevals = []
    for idx, score in sorteditems:
        fname = featurenames[idx]
        scorevals.append(round(score, 3))
        featurevals.append(featurenames[idx])
    results= {}
    for idx in range(len(featurevals)):
        results[featurevals[idx]]=scorevals[idx]  
    return results

In [ ]:
def sortedcoo(matrix):
    tuples = zip(matrix.col, matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
feature_names=countvector.get_feature_names()

In [ ]:
def getKeywords(id, doc):
    tfidfvector=transformer.transform(cv.transform([doc[id]]))
    sorteditems=sortedcoo(tfidfvector.tocoo())
    keywords=extractVector(feature_names,sorteditems,10)
    return keywords

In [ ]:
def printResults(id,keyword, df):
    # now print the results
    print("\n=====Title=====")
    print(df['title'][id])
    print("\n=====Abstract=====")
    print(df['abstract'][id])
    print("\n===Keywords===")
    for k in keyword:
        print(k,keyword[k])

In [ ]:
printResults(id,keywords, df)